In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/data/train.csv')
test = pd.read_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/data/test.csv')

sample = pd.read_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/data/sample_submission.csv')

In [64]:
print(train.shape, test.shape, sample.shape)

(18359, 14) (15021, 13) (15021, 2)


In [65]:
train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [66]:
train.isna().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
dtype: int64

In [67]:
train.head(2)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0


In [68]:
train.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                      int64
dtype: object

In [69]:
'gender',
'enrolled_university', 
'education_level',
'major_discipline', 
'experience', 
'company_size', 
'company_type',
'last_new_job'

'last_new_job'

In [70]:
train['city'].nunique()

123

In [71]:
train['gender'].nunique()

3

In [72]:
train['gender'].value_counts()

Male      12884
Female     1188
Other       189
Name: gender, dtype: int64

In [73]:
train['relevent_experience'].nunique()

2

In [74]:
train['enrolled_university'].nunique()

3

In [75]:
train['education_level'].nunique()

5

In [76]:
train['major_discipline'].nunique()

6

In [77]:
train['company_size'].nunique()

8

In [78]:
train['company_type'].nunique()

6

In [79]:
train.head(2)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0


In [80]:
train.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                      int64
dtype: object

In [81]:
train['last_new_job'].value_counts()

1        7567
>4       3339
2        2835
never    2186
4        1038
3        1027
Name: last_new_job, dtype: int64

In [82]:
train['experience'].value_counts()

>20    3437
5      1309
4      1250
3      1159
6      1125
2       992
9       979
10      967
7       950
8       755
15      695
11      667
14      602
16      549
12      497
1       452
<1      416
13      412
17      347
19      308
18      286
20      146
Name: experience, dtype: int64

 # CatBoost LGBM, XGBoost , RandomForest

# 1. Missing value imputation with mode

In [0]:
trainc = train.copy()

In [0]:
cols_with_missing = [col for col in train.columns 
                                 if train[col].isnull().any()]
for col in cols_with_missing:
  trainc[col] = trainc[col].fillna(trainc[col].mode()[0])                                

In [85]:
trainc.isna().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [86]:
test.head(2)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,16548,city_33,0.448,NaN,No relevent experience,Full time course,Graduate,STEM,<1,1000-4999,Public Sector,NaN,15
1,12036,city_28,0.939,Male,No relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,94


In [0]:
testc = test.copy()
cols_with_missing = [col for col in testc.columns 
                                 if testc[col].isnull().any()]

for col in cols_with_missing:
  testc[col] = testc[col].fillna(testc[col].mode()[0])

In [88]:
testc.isna().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

In [0]:
trainc = pd.get_dummies(trainc, columns = ['city','gender', 'relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type','last_new_job'])
testc = pd.get_dummies(testc, columns = ['city','gender', 'relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type','last_new_job'])

In [0]:
X_train = trainc.drop(['target'], axis=1)
y_train = trainc['target']
X_test = testc

In [91]:
sample.head(2)

,enrollee_id,target
0,16548,0
1,12036,0


## 1. RandomForest

In [0]:
from sklearn.ensemble import RandomForestClassifier

#build the classifier
clf = RandomForestClassifier()

#train the classifier
clf.fit(X_train, y_train)

#test the classifier
preds_rf = clf.predict(X_test)

sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = preds_rf

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub1.csv', index=None, header= True)

## 2. LightGBM

In [0]:
X_train = trainc.drop(['target'], axis=1)
y_train = trainc['target']
X_test = testc

In [0]:
import lightgbm as lgbm
d_train = lgbm.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 20
params['min_data'] = 20
params['max_depth'] = 20

clf = lgbm.train(params, d_train)

#Prediction
y_pred_lgbm = clf.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = y_pred_lgbm

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub3.csv', index=None, header= True)

In [0]:
import lightgbm as lgbm
d_train = lgbm.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.001
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'auc'
params['sub_feature'] = 0.8
params['num_leaves'] = 50
params['min_data'] = 10
params['max_depth'] = 50

clf = lgbm.train(params, d_train)

#Prediction
y_pred_lgbm1 = clf.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = y_pred_lgbm1

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub7.csv', index=None, header= True)

## 3.Neural Net

In [0]:
X_train = trainc.drop(['target'], axis=1)
y_train = trainc['target']
X_test = testc

In [0]:
print(X_train.shape, y_train.shape)

(18359, 187) (18359,)


In [0]:
#standardizing the input feature
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [0]:
from keras import Sequential
from keras.layers import Dense

In [0]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(100, activation='relu', kernel_initializer='random_normal', input_dim=187))
#Second  Hidden Layer
classifier.add(Dense(100, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [0]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [0]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=100)

Epoch 1/100
18359/18359 [==============================] - 3s 156us/step - loss: 0.3890 - accuracy: 0.8677
Epoch 2/100
18359/18359 [==============================] - 2s 134us/step - loss: 0.3726 - accuracy: 0.8678
Epoch 3/100
18359/18359 [==============================] - 2s 135us/step - loss: 0.3684 - accuracy: 0.8680
Epoch 4/100
18359/18359 [==============================] - 3s 137us/step - loss: 0.3637 - accuracy: 0.8679
Epoch 5/100
18359/18359 [==============================] - 2s 134us/step - loss: 0.3585 - accuracy: 0.8685
Epoch 6/100
18359/18359 [==============================] - 2s 132us/step - loss: 0.3512 - accuracy: 0.8687
Epoch 7/100
18359/18359 [==============================] - 2s 133us/step - loss: 0.3413 - accuracy: 0.8701
Epoch 8/100
18359/18359 [==============================] - 3s 136us/step - loss: 0.3296 - accuracy: 0.8712
Epoch 9/100
18359/18359 [==============================] - 2s 134us/step - loss: 0.3172 - accuracy: 0.8748
Epoch 10/100
18359/18359 [===========

In [0]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model


18359/18359 [==============================] - 0s 21us/step


[0.04211109913158402, 0.983495831489563]

In [0]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [0]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [ True]])

In [0]:
type(y_pred)
y_pred = pd.DataFrame(y_pred)
y_pred.head()

,0
0,False
1,False
2,False
3,False
4,False


In [0]:
y_pred.columns = ['target']

In [0]:
y_pred.head(2)

,target
0,False
1,False


In [0]:
y_pred['target'].replace(False, 0,inplace=True)
y_pred['target'].replace(True, 1,inplace=True)

In [0]:
y_pred['target'] = y_pred['target'].astype(int)
y_pred.head(2)

,target
0,0
1,0


In [0]:
sample['enrollee_id'] = test['enrollee_id']
sample['target'] = y_pred

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub4.csv', index=None, header= True)

## 4. XGBoost

In [0]:
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]

In [0]:
import xgboost as xgb

In [0]:
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

In [0]:
train_model1 = model1.fit(X_train, y_train)
train_model2 = model2.fit(X_train, y_train)

pred1 = train_model1.predict(X_test)
pred2 = train_model2.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = pred1

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub8.csv', index=None, header= True)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = pred2

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub9.csv', index=None, header= True)

In [0]:
#Let's do a little Gridsearch, Hyperparameter Tunning
model3 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = pred3
sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub10.csv', index=None, header= True)

In [112]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)

KeyboardInterrupt: ignored

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = pred4
sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub11.csv', index=None, header= True)

In [0]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model5 = gsearch2b.fit(X_trian, y_train)
pred5 = train_model5.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = pred5
sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub12.csv', index=None, header= True)

In [0]:
#Tune Gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model6 = gsearch3.fit(X_trian, y_train)
pred6 = train_model6.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = pred6
sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/sub13.csv', index=None, header= True)

## Lgbm Hyperparameter tuning

In [0]:
X_train = trainc.drop(['target'], axis=1)
y_train = trainc['target']
X_test = testc

In [0]:
import lightgbm as lgbm
d_train = lgbm.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.001
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'auc'
params['sub_feature'] = 0.8
params['num_leaves'] = 50
params['min_data'] = 10
params['max_depth'] = 50

clf = lgbm.train(params, d_train)

#Prediction
y_pred_lgbm1 = clf.predict(X_test)

In [0]:
import lightgbm as lgbm
d_train = lgbm.Dataset(X_train, label=y_train)

params = {}
params['learning_rate'] = 0.0001
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'roc_auc'
params['sub_feature'] = 0.1
params['num_leaves'] = 40
params['min_data'] = 5
params['max_depth'] = 100

clf = lgbm.train(params, d_train)

#Prediction
y_pred2 = clf.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = y_pred2

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/subb4.csv', index=None, header= True)

In [0]:
model = lgbm.LGBMClassifier( 
    boosting_type="gbdt",
    objective='binary',
    #is_unbalance=True, 
    random_state=10, 
    n_estimators=50,
    num_leaves=30, 
    max_depth=8,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15, 
    learning_rate=0.001,    
)

params_opt = {'n_estimators':range(200, 600, 80), 'num_leaves':range(20,60,10)}
gridSearchCV = GridSearchCV(estimator = model, 
    param_grid = params_opt, 
    scoring='roc_auc')
gridSearchCV.fit(X_train,y_train)

y_pred3 = gridSearchCV.predict(X_test)

In [0]:
sample['enrollee_id'] = X_test['enrollee_id']
sample['target'] = y_pred3

sample.to_csv('/content/drive/My Drive/Competitions - Neha/AV-JanataHack-HR Analytics/subb5.csv', index=None, header= True)